### 实现影像的裁剪、拼接
#### 1. 影像裁剪

In [3]:
from osgeo import gdal
import numpy as np


In [10]:
path_img = 'data/Section-2/s2_kunming_chenggong_4bands_10m.tif'


In [11]:
dset = gdal.Open(path_img)
geo_trans = dset.GetGeoTransform()            ### 地理转换参数
print('geotrans:', geo_trans)
x_min, y_max = geo_trans[0], geo_trans[3]     ### 左上角坐标
x_res, y_res = geo_trans[1], geo_trans[5]
x_max = geo_trans[0]+x_res*dset.RasterXSize
y_min = geo_trans[3]+y_res*dset.RasterYSize
print('extent:', x_min, x_max, y_min, y_max)


geotrans: (268170.0, 10.0, 0.0, 2765450.0, 0.0, -10.0)
extent: 268170.0 298130.0 2733210.0 2765450.0


In [12]:
### 设定裁剪范围
x_min_subs, x_max_subs = 277175.0, 286133.0
y_min_subs, y_max_subs = 2740200, 2750452.0


#### 计算裁剪范围对应图像行列号


In [13]:
col_start_subs = (x_min_subs - x_min)/x_res
row_start_subs = (y_max_subs - y_max)/y_res
print(row_start_subs, col_start_subs)


1499.8 900.5


In [14]:
### 裁剪影像起点行列号调整，及裁剪影像起点地理坐标调整
row_start_subs_update, col_start_subs_update = round(row_start_subs), round(col_start_subs)
print('new start point of the image coordinate:', row_start_subs_update, col_start_subs_update)
### 更新裁剪范围起点地理坐标
x_min_subs_update = x_min + col_start_subs_update*x_res
y_max_subs_update = y_max + row_start_subs_update*y_res
print(x_min_subs_update, y_max_subs_update)


new start point of the image coordinate: 1500 900
277170.0 2750450.0


#### 计算裁剪影像尺寸

In [15]:
x_size_subs = (x_max_subs-x_min_subs_update)/x_res
y_size_subs = -(y_max_subs_update-y_min_subs)/y_res
print(x_size_subs, y_size_subs)


896.3 1025.0


In [16]:
### 更新遥感影像尺寸
x_size_subs_update, y_size_subs_update = round(x_size_subs), round(y_size_subs)
print(x_size_subs_update, y_size_subs_update)


896 1025


#### 更新裁剪影像范围右下角点地理坐标

In [17]:
x_max_subs_update = x_min_subs_update + x_size_subs_update*x_res
y_min_subs_update = y_max_subs_update + y_size_subs_update*y_res
print(x_max_subs_update, y_min_subs_update)


286130.0 2740200.0


In [18]:
extent_subs_update = [x_min_subs_update, x_max_subs_update, y_min_subs_update, y_max_subs_update]  #
print('new extent:', extent_subs_update)


new extent: [277170.0, 286130.0, 2740200.0, 2750450.0]


#### 裁剪影像地理转换参数确定

In [19]:
geotrans_subs = [x_min_subs_update, x_res, 0, y_max_subs_update, 0, y_res]
geotrans_subs


[277170.0, 10.0, 0, 2750450.0, 0, -10.0]

#### 影像裁剪

In [20]:
### 获取裁剪影像数组
img_array = dset.ReadAsArray()
print(img_array.shape)          ### （波段数，行，列) -> (波段数,y,x)


(4, 3224, 2996)


In [21]:
### 获取裁剪影像数组
### 设定x,y方向上裁剪影像和原始影像左上角像元个数差异为0，709，获得裁剪影像数组
img_array_subs = img_array[:, 
                           row_start_subs_update:row_start_subs_update+y_size_subs_update, 
                           col_start_subs_update:col_start_subs_update+x_size_subs_update]
print(img_array_subs.shape)


(4, 1025, 896)


In [22]:
path_subs = 'data/Section-5/s2_kunming_chenggong_4bands_10m_subs.tif'
driver = gdal.GetDriverByName('GTiff')
dset_subs = driver.Create(path_subs, 
                        xsize = x_size_subs_update, 
                        ysize = y_size_subs_update, 
                        bands=dset.RasterCount, 
                        eType=gdal.GDT_Int16)
dset_subs.SetGeoTransform(geotrans_subs)
dset_subs.SetProjection(dset.GetProjection())
for i in range(dset.RasterCount):
    outband = dset_subs.GetRasterBand(i+1)       ### 获取波段1
    outband.WriteArray(img_array_subs[i])        ### 将np.array()数组写入波段1
dset_subs = None


思考：坚持不调整范围，可否进行裁剪？   
练习：设置不同子区域对10米分辨率影像进行裁剪，为后续影像拼接实验作准备。



#### 2. 影像拼接
数据准备：呈贡区两幅相同分辨率相同投影影像（理想情况）。


In [1]:
path_img_1 = 'data/Section-5/s2_kunming_chenggong_4bands_10m_subs.tif'
path_img_2 = 'data/Section-5/s2_kunming_chenggong_4bands_10m_subs_2.tif'


In [4]:
dset_subs1 = gdal.Open(path_img_1)
dset_subs2 = gdal.Open(path_img_2)
geo_trans_subs1 = dset_subs1.GetGeoTransform()        ### 地理转换参数
geo_trans_subs2 = dset_subs2.GetGeoTransform()        ### 地理转换参数
print('subs1:', geo_trans_subs1)
print('subs2:', geo_trans_subs2)
x_min_subs1 = geo_trans_subs1[0]
y_max_subs1 = geo_trans_subs1[3]
x_min_subs2 = geo_trans_subs2[0]
y_max_subs2 = geo_trans_subs2[3]
print(x_min_subs1, y_max_subs1)
print(x_min_subs2, y_max_subs2)


subs1: (277170.0, 10.0, 0.0, 2750450.0, 0.0, -10.0)
subs2: (273170.0, 10.0, 0.0, 2759450.0, 0.0, -10.0)
277170.0 2750450.0
273170.0 2759450.0


In [5]:
x_max_subs1 = x_min_subs1 + dset_subs1.RasterXSize * geo_trans_subs1[1]
y_min_subs1 = y_max_subs1 + dset_subs1.RasterYSize * geo_trans_subs1[5]
print('extent_subs1:', x_min_subs1, x_max_subs1, y_min_subs1, y_max_subs1)
x_max_subs2 = x_min_subs2 + dset_subs2.RasterXSize * geo_trans_subs2[1]
y_min_subs2 = y_max_subs2 + dset_subs2.RasterYSize * geo_trans_subs2[5]
print('extent_subs2:', x_min_subs2, x_max_subs2, y_min_subs2, y_max_subs2)


extent_subs1: 277170.0 286130.0 2740200.0 2750450.0
extent_subs2: 273170.0 280130.0 2748200.0 2759450.0


In [6]:
x_max_mosaic = max(x_max_subs1, x_max_subs2)
x_min_mosaic = min(x_min_subs1, x_min_subs2)
y_max_mosaic = max(y_max_subs1, y_max_subs2)
y_min_mosaic = min(y_min_subs1, y_min_subs2)
extent_mosaic = [x_min_mosaic, x_max_mosaic, y_min_mosaic, y_min_mosaic]
print('拼接影像范围：', extent_mosaic)


拼接影像范围： [273170.0, 286130.0, 2740200.0, 2740200.0]


In [7]:
### 定义拼接影像分辨率（采用拼接影像subs1分辨率）
x_res_mosaic, y_res_mosaic = geo_trans_subs1[1], geo_trans_subs1[5]
print(x_res_mosaic, y_res_mosaic)


10.0 -10.0


In [8]:
x_size_mosaic = (x_max_mosaic - x_min_mosaic)/x_res_mosaic
y_size_mosaic = abs((y_max_mosaic - y_min_mosaic)/y_res_mosaic)
print(x_size_mosaic, y_size_mosaic)


1296.0 1925.0


In [9]:
### 取整
x_size_mosaic, y_size_mosaic = int(x_size_mosaic), int(y_size_mosaic)


In [10]:
### 获得拼接影像地理转换参数
geotrans_mosaic = [x_min_mosaic, x_res_mosaic, 0, y_max_mosaic, 0, y_res_mosaic]
geotrans_mosaic


[273170.0, 10.0, 0, 2759450.0, 0, -10.0]

In [11]:
### 建立空数组
img_array_mosaic = np.zeros(shape=(dset_subs1.RasterCount, y_size_mosaic, x_size_mosaic))
img_array_mosaic.shape


(4, 1925, 1296)

In [14]:
### 待拼接影像起点在拼接影像上位置
### subs1
row_start_subs1 = int((y_max_subs1 - y_max_mosaic)/y_res_mosaic)
col_start_subs1 = int((x_min_subs1 - x_min_mosaic)/x_res_mosaic)
print(row_start_subs1, col_start_subs1)
img_array_mosaic[:, row_start_subs1:row_start_subs1+dset_subs1.RasterYSize, \
                 col_start_subs1:col_start_subs1+dset_subs1.RasterXSize] = dset_subs1.ReadAsArray()

### subs2
row_start_subs2 = int((y_max_subs2 - y_max_mosaic)/y_res_mosaic)
col_start_subs2 = int((x_min_subs2 - x_min_mosaic)/x_res_mosaic)
print(row_start_subs2, col_start_subs2)
img_array_mosaic[:, row_start_subs2:row_start_subs2+dset_subs2.RasterYSize, \
                 col_start_subs2:col_start_subs2+dset_subs2.RasterXSize] = dset_subs2.ReadAsArray()


900 400
0 0


#### 写出拼接影像

In [49]:
path_mosaic = 'data/Section-5/s2_kunming_chenggong_4bands_10m_subs_mosaic.tif'
driver = gdal.GetDriverByName('GTiff')
dset_mosaic = driver.Create(path_mosaic, 
                            xsize = x_size_mosaic, 
                            ysize = y_size_mosaic, 
                            bands=dset_subs1.RasterCount, 
                            eType=gdal.GDT_Int16)
dset_mosaic.SetGeoTransform(geotrans_mosaic)
dset_mosaic.SetProjection(dset_subs1.GetProjection())

for i in range(dset_subs1.RasterCount):
    outband = dset_mosaic.GetRasterBand(i+1)       ### 获取波段1
    outband.WriteArray(img_array_mosaic[i])        ### 将np.array()数组写入波段1
dset_mosaic = None
